#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 05 - Part 03 - How to use Docker?</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

### Example "hello world" in Docker
* Get the image for `hello-world` from Docker Hub:
  
  ```sh
 $ docker pull hello-world
Using default tag: latest
latest: Pulling from library/hello-world
0e03bdcc26d7: Pull complete
Digest: sha256:6a65f928fb91fcfbc963f7aa6d57c8eeb426ad9a20c7ee045538ef34847f44f1
Status: Downloaded newer image for hello-world:latest
docker.io/library/hello-world:latest

  ```

* Create a container which runs, prints a message, then stops:

```sh
$ docker run hello-world
```
```text
Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

```

* We can see downloaded images:

```sh
$ docker image ls
```
```
REPOSITORY               TAG                               IMAGE ID            CREATED             SIZE
hello-world              latest                            bf756fb1ae65        4 months ago        13.3kB
nvidia/cuda              10.1-cudnn7-runtime-ubuntu16.04   e11e11484e2e        5 months ago        1.71GB
ubuntu                   18.04                             a2a15febcdf3        9 months ago        64.2MB
ubuntu                   latest                            a2a15febcdf3        9 months ago        64.2MB
python                   3.7                               e497dabd8450        9 months ago        918MB
nvidia/cuda              10.1-runtime-ubuntu16.04          78d4afa83d9f        10 months ago       1.2GB
nvidia/cuda              latest                            89bd89411ed6        10 months ago       2.76GB
ubuntu                   16.04                             5e13f8dd4c1a        10 months ago       120MB
continuumio/miniconda3   latest                            6b5cf97566c3        12 months ago       457MB
```

* And running container (with `-a` since `hello-world` has stopped already):

```sh
$ docker container ls -a
```
```
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                     PORTS               NAMES
96e7d48c0ddb        hello-world         "/hello"            2 minutes ago       Exited (0) 2 minutes ago                       jolly_poincare
```

* Images will contain the libraries needed to run, for `hello-world` this is nothing so is really small
* The Dockerfile for creating the image is as small as it gets:
```dockerfile
FROM scratch
COPY hello /
CMD ["/hello"]
```
* States the image will be made from scratch, copy the executable `hello` to the root directory, and the command to run when the container is run is `/hello`

* Images have internal file storage for storing libraries, data, executables, etc.
* Files are stored on the host's file system rather than an internal virtualized hard drive with a file system
* OS requests (IO, interrupts, any other hardware access) go to the host OS via the container daemon
* Each container appears internally to be an independent Linux system but is actually using the host OS 
* Segregation between containers is enforced by container daemon
* Compare to a virtual machine which is presented with a virtual hardware environment and must include everything above that (file system, OS, etc.)

* Another example: let's play with the Python base image
```sh
$ docker run -ti --rm python bash
```
```
Unable to find image 'python:latest' locally
latest: Pulling from library/python
376057ac6fa1: Pull complete
5a63a0a859d8: Pull complete
496548a8c952: Pull complete
2adae3950d4d: Pull complete
0ed5a9824906: Pull complete
bb94ffe72389: Pull complete
241ada007777: Pull complete
be68aa7d1eeb: Pull complete
820ffc2e28ca: Pull complete
Digest: sha256:6fcd27ebeb1a5b4fd289ff15cb666e619c060c7b76f5a1b1a99d7cddb6de337a
Status: Downloaded newer image for python:latest
```

* This will pull `python` from Docker Hub automatically if not already pulled
* `-ti --rm` means allocate a pseudo-TTY, interactive mode, and remove the container when completed
* `bash` means use `bash` as the command to run when starting the container, this is so we can interact with the container
* Also pinning the image version for Python 3.7, more consistent than just `python`

```sh
$ docker run -ti --rm python:3.7 bash
```
```
root@a5b78615711c:/# cat /etc/os-release
PRETTY_NAME="Debian GNU/Linux 10 (buster)"
NAME="Debian GNU/Linux"
VERSION_ID="10"
VERSION="10 (buster)"
VERSION_CODENAME=buster
ID=debian
HOME_URL="https://www.debian.org/"
SUPPORT_URL="https://www.debian.org/support"
BUG_REPORT_URL="https://bugs.debian.org/"
```

* Image for `python:3.7` is based on Debian 10 apparently
* If run without `bash`, the command stated in the Dockerfile is used instead, which is `python`:

```sh
$ docker run -ti --rm python:3.7
```
```
Python 3.7.4 (default, Aug 14 2019, 12:09:51)
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>>
```
* If run without `-ti --rm` the container will silently run `python` then exit

* Let's create our own Pythonic version of `hello-world`
* Create a script `hello.py` that just has `print("Hello, Docker!")`
* Create a Dockerfile:

```dockerfile
from python:3.7
COPY hello.py /
CMD ["python","hello.py"]
```
* This will copy `hello.py` into the root of the image's files and set the command to run `python` on startup with the given script file

* Build the image, specifying the Dockerfile and tagging it as "hello-python":

```sh
$ docker build -f Dockerfile -t hello-python .
```
```
Sending build context to Docker daemon  79.87kB
Step 1/3 : from python
 ---> 659f826fabf4
Step 2/3 : COPY hello.py /
 ---> 35372c5a881a
Step 3/3 : CMD ["python","hello.py"]
 ---> Running in 9948be278156
Removing intermediate container 9948be278156
 ---> 5d1337963697
Successfully built 5d1337963697
Successfully tagged hello-python:latest
```

* Run the image:

```sh
$ docker run --rm hello-python
Hello Docker!
```

* We can see what files are present by running the container interactively:

```sh
$ docker run -ti --rm hello-python bash
root@279d520691f9:/# ls
bin  boot  dev  etc  hello.py  home  lib  lib64  media  mnt  opt  proc  root  run  sbin  srv  sys  tmp  usr  var
```

* Slightly more involved example: web service hosted with Flask
* Define a simple server in `hello_flask.py` which unsurprisingly responds with "Hello, World!":

```python
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'
```

* Dockerfile is quite similar:

```dockerfile
from python:3.7
RUN pip install flask

ENV FLASK_APP hello_flask.py
EXPOSE 5000

COPY hello_flask.py /

CMD ["flask","run","--host=0.0.0.0"]
```

* `RUN` runs a command in the image
* `ENV` creates an environment variable in the image
* `EXPOSE` documents that port 5000 is expected to be listened to by the application

* When this application is run the port 5000 has to be explicitly exposed:

```sh
$ docker run -d -p 5000:5000 hello-flask
b65b10374ef362934b21108c393efbd1c7fd2818809f7df3e38439295683e2b8
```

* `-d` means run detached in the background
* Can now query our running web service:

```sh
$ curl localhost:5000
Hello, World!
```

* `--net=host` can also be used to connect the host's network with the container's so individual ports don't need to be specified

* Dockerfile scripts setup images step-by-step
* Each step is cached by Docker so that changes to the file don't cause slow and expensive operations to be rerun without necessity
* Commands are run in the image as if it was a running Linux system
* Once the script is run the state of the system at that point in time becomes the image
* `CMD` specifies the command to run when the container is launcher, be aware this is run as root within the container by default

# That's it!

On to practicals...